# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here

import pandas as pd
import numpy as np


cols= ['target', 'ids', 'date', 'flag', 'user', 'text']

df = pd.read_csv('training.csv', encoding='latin-1')

df.columns = cols

df.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [2]:
data = df.sample(20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
# your code here

import re

def clean_up(s): 
    element = re.sub('http://.+', '', s)
    element = re.sub('[^a-zA-Z0-9]', ' ', element)
    return (re.sub('\d+',' ',element)).lower()

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokenize(s):
    return word_tokenize(s)


from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # get returns second argument if first key does not exist


def lemmatize(l):
  
    lem = WordNetLemmatizer()
    lemmatized = [lem.lemmatize(w,get_wordnet_pos(w)) for w in l]
    
    return lemmatized

from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(l):
    
    filtered_sentence = []
    
    for w in l:
        if w not in stopwords.words('english'):
            filtered_sentence.append(w)
    
    return filtered_sentence

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data['text_processed'] = data['text'].apply(clean_up)
data.head()

,target,ids,date,flag,user,text,text_processed
286974,0,1994059119,Mon Jun 01 11:20:56 PDT 2009,NO_QUERY,toadrn,@marybeth494 at the Drs office waiting to be t...,marybeth at the drs office waiting to be tol...
629369,0,2231623384,Thu Jun 18 18:48:23 PDT 2009,NO_QUERY,tiffanystacy,I'm so bored now,i m so bored now
513356,0,2190365530,Tue Jun 16 02:31:12 PDT 2009,NO_QUERY,annasaccone,@amybabydoll69 @hollymae20 neither can i it's...,amybabydoll hollymae neither can i it s n...
385758,0,2053670353,Sat Jun 06 05:20:08 PDT 2009,NO_QUERY,SweetieE94,is still ill,is still ill
944313,4,1795505240,Thu May 14 07:53:24 PDT 2009,NO_QUERY,iShae86,good morning twittees! hope u all have a great...,good morning twittees hope u all have a great...


In [5]:
data['text_processed'] = data['text_processed'].apply(tokenize)
data.head()

,target,ids,date,flag,user,text,text_processed
286974,0,1994059119,Mon Jun 01 11:20:56 PDT 2009,NO_QUERY,toadrn,@marybeth494 at the Drs office waiting to be t...,"[marybeth, at, the, drs, office, waiting, to, ..."
629369,0,2231623384,Thu Jun 18 18:48:23 PDT 2009,NO_QUERY,tiffanystacy,I'm so bored now,"[i, m, so, bored, now]"
513356,0,2190365530,Tue Jun 16 02:31:12 PDT 2009,NO_QUERY,annasaccone,@amybabydoll69 @hollymae20 neither can i it's...,"[amybabydoll, hollymae, neither, can, i, it, s..."
385758,0,2053670353,Sat Jun 06 05:20:08 PDT 2009,NO_QUERY,SweetieE94,is still ill,"[is, still, ill]"
944313,4,1795505240,Thu May 14 07:53:24 PDT 2009,NO_QUERY,iShae86,good morning twittees! hope u all have a great...,"[good, morning, twittees, hope, u, all, have, ..."


In [6]:
data['text_processed'] = data['text_processed'].apply(lemmatize)
data.head()

,target,ids,date,flag,user,text,text_processed
286974,0,1994059119,Mon Jun 01 11:20:56 PDT 2009,NO_QUERY,toadrn,@marybeth494 at the Drs office waiting to be t...,"[marybeth, at, the, drs, office, wait, to, be,..."
629369,0,2231623384,Thu Jun 18 18:48:23 PDT 2009,NO_QUERY,tiffanystacy,I'm so bored now,"[i, m, so, bore, now]"
513356,0,2190365530,Tue Jun 16 02:31:12 PDT 2009,NO_QUERY,annasaccone,@amybabydoll69 @hollymae20 neither can i it's...,"[amybabydoll, hollymae, neither, can, i, it, s..."
385758,0,2053670353,Sat Jun 06 05:20:08 PDT 2009,NO_QUERY,SweetieE94,is still ill,"[be, still, ill]"
944313,4,1795505240,Thu May 14 07:53:24 PDT 2009,NO_QUERY,iShae86,good morning twittees! hope u all have a great...,"[good, morning, twittees, hope, u, all, have, ..."


In [7]:
data['text_processed'] = data['text_processed'].apply(remove_stopwords)
data.head()

,target,ids,date,flag,user,text,text_processed
286974,0,1994059119,Mon Jun 01 11:20:56 PDT 2009,NO_QUERY,toadrn,@marybeth494 at the Drs office waiting to be t...,"[marybeth, drs, office, wait, told, bp, still,..."
629369,0,2231623384,Thu Jun 18 18:48:23 PDT 2009,NO_QUERY,tiffanystacy,I'm so bored now,[bore]
513356,0,2190365530,Tue Jun 16 02:31:12 PDT 2009,NO_QUERY,annasaccone,@amybabydoll69 @hollymae20 neither can i it's...,"[amybabydoll, hollymae, neither, fun, apply, t..."
385758,0,2053670353,Sat Jun 06 05:20:08 PDT 2009,NO_QUERY,SweetieE94,is still ill,"[still, ill]"
944313,4,1795505240,Thu May 14 07:53:24 PDT 2009,NO_QUERY,iShae86,good morning twittees! hope u all have a great...,"[good, morning, twittees, hope, u, great, day]"


In [15]:
data['target'].value_counts()

4    10063
0     9937
Name: target, dtype: int64

In [22]:
data['target'] = data['target'].apply(lambda x: True if x == 4 else False)

In [23]:
data.head()

,target,ids,date,flag,user,text,text_processed
286974,False,1994059119,Mon Jun 01 11:20:56 PDT 2009,NO_QUERY,toadrn,@marybeth494 at the Drs office waiting to be t...,"[marybeth, drs, office, wait, told, bp, still,..."
629369,False,2231623384,Thu Jun 18 18:48:23 PDT 2009,NO_QUERY,tiffanystacy,I'm so bored now,[bore]
513356,False,2190365530,Tue Jun 16 02:31:12 PDT 2009,NO_QUERY,annasaccone,@amybabydoll69 @hollymae20 neither can i it's...,"[amybabydoll, hollymae, neither, fun, apply, t..."
385758,False,2053670353,Sat Jun 06 05:20:08 PDT 2009,NO_QUERY,SweetieE94,is still ill,"[still, ill]"
944313,True,1795505240,Thu May 14 07:53:24 PDT 2009,NO_QUERY,iShae86,good morning twittees! hope u all have a great...,"[good, morning, twittees, hope, u, great, day]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [8]:
# your code here

from nltk.probability import FreqDist
import itertools


blob = data['text_processed'].tolist()

blob = itertools.chain.from_iterable(blob)

fdist = FreqDist(blob)

common5000 = dict(fdist.most_common(5000))

most_common = list(common5000.keys())

most_common

['get',
 'go',
 'day',
 'work',
 'good',
 'like',
 'love',
 'u',
 'quot',
 'well',
 'time',
 'today',
 'one',
 'miss',
 'lol',
 'know',
 'back',
 'see',
 'want',
 'think',
 'really',
 'make',
 'feel',
 'im',
 'amp',
 'still',
 'night',
 'come',
 'new',
 'home',
 'watch',
 'oh',
 'thanks',
 'need',
 'na',
 'hope',
 'last',
 'much',
 'tomorrow',
 'look',
 'morning',
 'great',
 'twitter',
 'say',
 'sleep',
 'wish',
 'wait',
 'take',
 'haha',
 'sad',
 'bad',
 'week',
 'fun',
 'try',
 'sorry',
 'happy',
 'right',
 'friend',
 'tonight',
 'would',
 'thing',
 'way',
 'though',
 'even',
 'nice',
 'yeah',
 'gon',
 'hate',
 'bed',
 'could',
 'tweet',
 'follow',
 'show',
 'yes',
 'start',
 'hour',
 'people',
 'school',
 'soon',
 'guy',
 'let',
 'next',
 'weekend',
 'first',
 'girl',
 'life',
 'cant',
 'dont',
 'thank',
 'hey',
 'play',
 'awesome',
 'year',
 'little',
 'tire',
 'use',
 'x',
 'long',
 'ok',
 'find',
 'rain',
 'wan',
 'everyone',
 'help',
 'sick',
 'lt',
 'give',
 'eat',
 'something'

In [31]:
target = data['target']

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [9]:
# your code here


for_review = data['text_processed']

def find_features(document):
    words = set(document)
    features = {}
    for w in most_common:
        features[w] = (w in words)
    
    return features

X = for_review.map(find_features)


In [32]:
target_list = target.tolist()

In [33]:
target_list[0:10]

[False, False, False, False, True, True, True, False, True, False]

In [17]:
list_x = X.values.tolist()


In [34]:
list_x[0:10]

[{'get': False,
  'go': False,
  'day': False,
  'work': False,
  'good': False,
  'like': False,
  'love': False,
  'u': False,
  'quot': False,
  'well': False,
  'time': False,
  'today': False,
  'one': False,
  'miss': False,
  'lol': False,
  'know': False,
  'back': False,
  'see': False,
  'want': False,
  'think': False,
  'really': False,
  'make': False,
  'feel': False,
  'im': False,
  'amp': False,
  'still': True,
  'night': False,
  'come': False,
  'new': False,
  'home': False,
  'watch': False,
  'oh': False,
  'thanks': False,
  'need': False,
  'na': False,
  'hope': False,
  'last': False,
  'much': False,
  'tomorrow': False,
  'look': False,
  'morning': False,
  'great': False,
  'twitter': False,
  'say': False,
  'sleep': False,
  'wish': False,
  'wait': True,
  'take': False,
  'haha': False,
  'sad': False,
  'bad': False,
  'week': False,
  'fun': False,
  'try': False,
  'sorry': False,
  'happy': False,
  'right': False,
  'friend': False,
  'tonight': 

In [35]:
X_total = []

for element in range(0,5001):
    X_total.append((list_x[element],target_list[element]))

In [38]:
len(X_total[0])

2

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [63]:
# your code here

import nltk

training_set = X_total[:3000]

testing_set = X_total[3000:]


In [62]:
len(X_total)

5001

In [64]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [65]:
classifier.show_most_informative_features(15)

Most Informative Features
                     sad = True            False : True   =     38.2 : 1.0
                    lose = True            False : True   =     17.0 : 1.0
                    suck = True            False : True   =     16.3 : 1.0
                    bore = True            False : True   =     10.3 : 1.0
                    hurt = True            False : True   =      9.5 : 1.0
                    hate = True            False : True   =      8.8 : 1.0
                    sick = True            False : True   =      7.8 : 1.0
           unfortunately = True            False : True   =      7.6 : 1.0
               goodnight = True             True : False  =      7.0 : 1.0
                   wrong = True            False : True   =      6.2 : 1.0
                   thank = True             True : False  =      5.9 : 1.0
                    miss = True            False : True   =      5.8 : 1.0
                    room = True            False : True   =      5.8 : 1.0

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [66]:
# your code here

nltk.classify.accuracy(classifier, testing_set)

0.6796601699150425

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here